In [53]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
# Load .env enviroment variables into the notebook
load_dotenv()
# Get the postgres connection information from os file. 

DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

In [35]:
real_acct_df = pd.read_csv('Real_acct_owner/real_acct.txt', sep='\t',lineterminator='\r',header=0,encoding='latin1',low_memory=False)

In [61]:
new_cols = ['acct', 'yr', 'mailto', 'mail_addr_1', 'mail_city',
       'mail_state', 'mail_zip', 'mail_country', 'str_pfx', 'str_num',
       'str_num_sfx', 'str', 'str_sfx', 'str_sfx_dir', 'str_unit',
       'site_addr_1', 'site_addr_2', 'site_addr_3','bld_val', 'x_features_val', 'ag_val', 'assessed_val', 'tot_appr_val',
       'tot_mkt_val', 'prior_land_val', 'prior_bld_val',
       'prior_x_features_val', 'prior_ag_val', 'prior_tot_appr_val',
       'prior_tot_mkt_val', 'new_construction_val', 'tot_rcn_val',
       'value_status', 'certified_date']

In [63]:
# remove the columns that we don't need
real_acct_df = real_acct_df[new_cols]
real_acct_df.head()

,acct,yr,mailto,mail_addr_1,mail_city,mail_state,mail_zip,mail_country,str_pfx,str_num,...,prior_land_val,prior_bld_val,prior_x_features_val,prior_ag_val,prior_tot_appr_val,prior_tot_mkt_val,new_construction_val,tot_rcn_val,value_status,certified_date
0,\n0010010000013,2021.0,CITY OF HOUSTON,PO BOX 1562,HOUSTON,TX,77251-1562,,,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Value Unchanged, Notice Not Required",08/06/2021
1,\n0010020000001,2021.0,CURRENT OWNER,1717 SAINT JAMES PLACE STE 112,HOUSTON,TX,77056-3412,,,907.0,...,300060.0,7613.0,0.0,0.0,307673.0,307673.0,0.0,0.0,Noticed,NaN
2,\n0010020000003,2021.0,MILBY CHARLES FAMILY PTNSH,2612 TODVILLE RD,SEABROOK,TX,77586-3008,,,0.0,...,860406.0,0.0,24360.0,0.0,884766.0,884766.0,0.0,0.0,Noticed,07/02/2021
3,\n0010020000004,2021.0,"FEAGIN MICHAEL RYAN TRUST % LYNDA EZER, TRUSTEE",10038 DEL MONTE DR,HOUSTON,TX,77042-2432,,,0.0,...,430203.0,0.0,12180.0,0.0,442383.0,442383.0,0.0,0.0,Noticed,07/02/2021
4,\n0010020000013,2021.0,BUFFALO BAYOU PARTNERSHIP,1019 COMMERCE ST STE 200,HOUSTON,TX,77002-1701,,,921.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Value Unchanged, Notice Not Required",08/06/2021


In [64]:
# remove the \n from the account column
def clean(x):
    x = x.replace('\n','')
    return x

In [65]:
real_acct_df['acct'] = real_acct_df['acct'].apply(clean)

In [66]:
real_acct_df.head(2)

,acct,yr,mailto,mail_addr_1,mail_city,mail_state,mail_zip,mail_country,str_pfx,str_num,...,prior_land_val,prior_bld_val,prior_x_features_val,prior_ag_val,prior_tot_appr_val,prior_tot_mkt_val,new_construction_val,tot_rcn_val,value_status,certified_date
0,0010010000013,2021.0,CITY OF HOUSTON,PO BOX 1562,HOUSTON,TX,77251-1562,,,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Value Unchanged, Notice Not Required",08/06/2021
1,0010020000001,2021.0,CURRENT OWNER,1717 SAINT JAMES PLACE STE 112,HOUSTON,TX,77056-3412,,,907.0,...,300060.0,7613.0,0.0,0.0,307673.0,307673.0,0.0,0.0,Noticed,NaN


In [67]:
# load 5 million rows to local database
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:5432/Houston_real_estate')
# load df to database
new_hcad_df.to_sql('real_acct',engine,index=False,if_exists='replace')